Please remember to update the token. For multiplayer to work, each player needs a different token. Please feel free to use multiple tokens in the same group.

In [1]:
#Update your token
STUDENT_TOKEN = 'MANUEL SAN ROMÁN GARCÍA'

There should be no need to change the code below.

In [2]:
## ignore this code, just used for submission
import requests
import pprint
import json
import random
import time
from copy import copy, deepcopy

class Game:
  def __init__(self, state, status, player):
    self.state = state
    self.status = status
    self.player = player

  def is_waiting(self):
    return self.status == 'waiting'

  def is_end(self):
    return self.status == 'complete'
  
  def get_winner(self):
    return None

  def actions(self):
    return []

  def print(self):
    print(self.state)

def new_game(game_type, multi_player = False):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/new-game?TOKEN=%s&game-type=%s&multi-player=%s' % (STUDENT_TOKEN, game_type, 'True' if multi_player else 'False'))
    if r.status_code == 200:
      return r.json()['game-id']
    print(r.content)

def join_game(game_type, game_id):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/join-game?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return r.json()['player']
    print(r.content)

def game_state(game_type, game_id, GameClass):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/game-state?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return GameClass(r.json()['state'], r.json()['status'], r.json()['player'])
    print(r.content)

def update_game(game_type, game_id, player, move):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/update-game?TOKEN=%s&game-type=%s&game-id=%s&player=%s&move=%s' % (STUDENT_TOKEN, game_type, game_id, player, move))
    if r.status_code == 200:
      return r.content
    print(r.content)

def game_loop(solver, GameClass, game_type, multi_player = False, id = None):
  while id == None:
    print('Creating new game...')
    id = new_game(game_type, multi_player)

  print('Joining game with id: %s' % id)
  player = join_game(game_type, id)

  print('Playing as %s' % player)

  game = game_state(game_type, id, GameClass)
  print('Waiting for the other player to join...')
  while game.is_waiting():
    time.sleep(10)
    game = game_state(game_type, id, GameClass)

  while True:
    game = game_state(game_type, id, GameClass)
    game.print_game()
    if game.is_end():
      if game.player == '-':
        print('Server says there is a draw')
      else:
        print('Server says your agent ' + ('WON' if game.player == player else 'LOST'))
      return
    if game.player == player:
      print('Making next move...')
      move = solver(game, actions_of_game)
      print(move)
      update_result = update_game(game_type, id, player, json.dumps(move))
      print(update_result)
    else:
      time.sleep(2)

There is almost certainly no need to change this code, but if you must you can.

In [3]:
from functools import reduce
from copy import copy, deepcopy
import json
import random

class GinRummy(Game):
  def __init__(self, state, status, player):
    Game.__init__(self, state, status, player)
    self.l = []

  def past_actions(self):
    self.l.append(self.state['discard'])
    return self.l

  def actions(self):
    return [] # this should return the possible actions

  def get_winner(self):
    return '.' # this should return the actual winner

  def other_player(self):
    if self.player == 'O': return 'X'
    if self.player == 'X': return 'O'

  def letter_to_symbol(letter):
    if letter == 'S': return '\u2664'
    if letter == 'H': return '\u2665'
    if letter == 'D': return '\u2666'
    if letter == 'C': return '\u2667'
    return letter

  def print_game(self):
    if self.player == 'O':
      print('X: %i, \033[1mO: %i\033[0m' % tuple(self.state['score']))
    else:
      print('\033[1mX: %i\033[0m, O: %i' % tuple(self.state['score']))
  
    for card in self.get_hand():
      print('%i%s' % (card[0], GinRummy.letter_to_symbol(card[1])), end = ' ')
    print()

    if self.state['discard']:
      print('Discard: %i%s' % (self.state['discard'][0], GinRummy.letter_to_symbol(self.state['discard'][1]) ))
    else:
      print('Discard: empty')


  def get_hand(self):
    return self.state['hand']

  def get_hand_state(self):
    return self.state['hand_state']

Instead of changing this code for a random solver, write new code that also returns the next move. For example, create a new "amazing_solver".

Solvers need to return lists:
*   If the hand status is 'pick', it needes to be ['deck'] or ['discard'] 
*   If the hand status is 'discard', it needs to be the action (discard, knock, gin), and the value and suit of the card to discard (there is always a discard card). E.g. ['gin', 4, 'H']

In [4]:
def random_solver(game):
  hand_state = game.get_hand_state()

  if hand_state == 'pick':
    return ['deck'] if random.randint(0, 1) == 0 else ['discard']
  else:
    hand = game.get_hand()
    card = hand[random.randint(0, len(hand) - 1)]
    return ['discard', card[0], card[1]]

This code is the one that starts the game. In different games you will need to change the class (GinRummy is the one for gin rummy game).

The 'multi_player' variable can be True or False. If you set multi_palyer to True, you will need to provide a game_id against which to play or if using None a new game will be created (which another player can join).

In [5]:
from ast import Num
from queue import PriorityQueue

# We create a list to keep our past action and the other player's past actions.
# We also add imposible cards to keep the algorithm from looking for impossible melds.
actions_of_game = [(0, "H"), (0, "C"), (0, "D"), (0, "S"), (13, "H"), (13, "C"), (13, "D"), (13, "S")] 


class Hand: # Class to create object for each move
  def __init__(self, game, actions_of_game): 
    self.hand = game.get_hand()
    self.ordered_hand = self.to_tuple() # List of tuples
    self.melds = self.get_melds()
    self.non_melds = self.get_non_melds()
    if game.state['discard']: # If there is a discard from the other player (if it's not first move), append that card.
      self.discard_on_table = ((game.state['discard'][0], game.state['discard'][1]))
      actions_of_game.append(self.discard_on_table)
    self.past_actions = actions_of_game # Past actions during game loop


  def to_tuple(self): # We create another attribute to order hand and change cards (lists) to tuples.
    hand = self.hand
    tuples = PriorityQueue()
    ordered_cards = []

    for card in hand:
      tup = ((card[0], card[1]))
      tuples.put(tup)

    while not tuples.empty():
      next_item = tuples.get()
      ordered_cards.append(next_item)

    return ordered_cards

  
  def get_consecutive (self, x, hand): # We get melds from consecutive cards
    suits = ['C', 'D', 'S', 'H'] # List to loop over suits
    different_suits = []
    melds = []

    for suit in suits:
      same_suit = [] # We append each card in its corresponding list depending on its suit.
      for card in hand:
        card_suit = card[1]
        if card_suit == suit:
          same_suit.append(card)
      different_suits.append(same_suit)

    for i in range(len(different_suits)): # Loop to create melds
        if len(different_suits[i]) != 0:
          consecutive = set() # Create set to not repeat cards
          for j in range(len(different_suits[i])):
            if different_suits[i][j][0] == different_suits[i][j-1][0] + 1: # If actual card value equals previous card value, add both cards to set
              consecutive.add(different_suits[i][j])  
              consecutive.add(different_suits[i][j-1])
            else: # If the two cards are not consecutive, create a new empty set
              consecutive = set()

          if len(consecutive) >= x: # If len of meld is > x, sort list and append to main list with all consecutive melds
            consecutive = sorted(consecutive, key=lambda tup: tup[0])
            melds.append(consecutive)

    return melds


  def get_same(self, x, hand): # Get melds of same number
    melds = []
    nums = { # Create a dictionary to store all cards according to its value
        1: [],
        2: [],
        3: [],
        4: [],
        5: [],
        6: [],
        7: [],
        8: [],
        9: [],
        10: [],
        11: [],
        12: [],
        13: []
    }

    for card in hand: # We append each card to its corresponding number
      for num in nums:
        if card[0] == num:
          nums[card[0]].append(card)

    for num in nums:
      if len(nums[num]) >= x: # If len of list of each number > x, sort list, and append it to main list of same number melds.
        nums[num] = sorted(nums[num])
        melds.append(nums[num])

    return melds


  def get_melds(self): # Get all melds for actual hand
  # As it is easier to get a consecutive card, we run the code to get consecutive melds before the code of same cards
    hand = deepcopy(self.ordered_hand)
    all_melds = []

    melds_consecutive = self.get_consecutive(3, hand) # Get melds of consecutive cards

    for meld_c in melds_consecutive:
      all_melds.append(meld_c)
      for card in meld_c:
        hand.remove(card) # Remove the cards from the hand so that a card is not used in two melds.


    melds_same = self.get_same(3, hand) # Get melds of same cards

    for meld_s in melds_same: # We append cards of this type of melds to a list with all melds.
      all_melds.append(meld_s)
      for card in meld_s:
        hand.remove(card) # Remove the cards from the hand so that a card is not used in two melds.
    return all_melds

  def get_non_melds(self): # We get the card that are not in a meld (deadwood)
    meldCards = []
    nonMeldCards = []

    # Store all the cards from the various melds into a single array
    for meld in self.melds:
      for card in meld:
        meldCards.append(card)

    # Store all the cards that are not in a meld in another list
    for card in self.ordered_hand:
      if card not in meldCards:
        nonMeldCards.append(card)
    
    return nonMeldCards


  def deck_or_discard(self): # Choose the card on discard deck or the other deck.
    discard = deepcopy(self.discard_on_table)
    discard = (discard[0], discard[1])
    hand = deepcopy(self.ordered_hand)

    hand.append(discard) # We append the discard card to simulate possible melds with that card
    hand = sorted(hand)

    all_melds = []

    # Create melds taking into account that discard card is inside the simulated hand.
    melds_consecutive = self.get_consecutive(2, hand) # x = 2, because we can make a pair

    for meld_c in melds_consecutive:
      all_melds.append(meld_c)
      for card in meld_c:
        hand.remove(card)


    melds_same = self.get_same(2, hand)

    for meld_s in melds_same:
      all_melds.append(meld_s)
      for card in meld_s:
        hand.remove(card)

    variable = False 
    for meld in all_melds: # If the discard card is in any possible 2-card meld, then return true, we pick the card from discard
      if discard in meld:
        variable = True # We pick the card from discard

      # Else we pick the card from deck
    
    return variable


  def decide_discard(self, game): # Get the card we are going to discard based on heruistics.
    non_melds = deepcopy(self.non_melds)
    non_melds_initial = deepcopy(self.non_melds)
    melds = deepcopy(self.melds)

    all_consecutive_melds = []

    non_melds_check = deepcopy(non_melds)
    melds_consecutive = self.get_consecutive(2, non_melds_check) # Melds_consecutive here represents 2 consecutive cards that are not in a meld

    for meld_c in melds_consecutive: # We loop trough the melds _consecutive just to remove it from non_melds as it is in a pair or two-consecutives
      all_consecutive_melds.append(meld_c) # Append possible 2-card melds in a new list.
      for card in meld_c:
        non_melds_check.remove(card)
    
    for meld in all_consecutive_melds: # Loop to decide if a consecutive non-meld card is good to discard.
      number_down = (meld[0][0]-1, meld[0][1]) # Smaller card for the current possible future meld.
      number_up = (meld[-1][0]+1, meld[-1][1]) # Greater card for the current possible future meld.
      if number_down in self.past_actions and number_up in self.past_actions: # If both up and down cards for that suit are not going to appear anymore
        pass # We keep that card in non_melds list as it is a candidate to discard.
      else: # If at least one of them is not discarded, the cards between these two are not candidates to discard, so we remove them.
        non_melds.remove(meld[0])  
        non_melds.remove(meld[-1])
        all_consecutive_melds.remove(meld) # We remove the non_meld as it is an empty meld.


    all_same_melds = []

    non_melds_check = deepcopy(non_melds)
    melds_same = self.get_same(2, non_melds_check) # We repeat the previous methodology but for the cards with the same number.

    for meld_s in melds_same:
      all_same_melds.append(meld_s)
      for card in meld_s:
        non_melds_check.remove(card)

    past_actions = deepcopy(self.past_actions)
    while all_same_melds != []: # While there are possible melds of same cards
      same_number = all_same_melds[0][0][0]
      count = 0 # Counter to check how many cards with the same value are in past actions.
      for action in past_actions: # For each card in past action, if this card has the same value as the one for each 2-card possible meld
        if same_number == action[0]:
          count += 1
      if count >= 2: # If there are two cards with the same value in past actions, then we cannot create a meld, so discard card.
        all_same_melds.remove(all_same_melds[0])
      else: # If there is only one card, or no cards in past actions with the same number, we remove those cards from non_melds as they are not candidates to discard
        non_melds.remove(all_same_melds[0][0])
        non_melds.remove(all_same_melds[0][-1])
        all_same_melds.remove(all_same_melds[0] )# We remove the non_meld as it is an empty meld.
          

    non_melds = sorted(non_melds, key=lambda tup: tup[0])
    if len(non_melds) > 0: # If there are candidates to discard, return the highest card.
      return non_melds[-1]
    else:
      if len(non_melds_initial) > 0: # If there are not candiates to discard, as all the cards are possible future melds, return the highest non_meld card
        return non_melds_initial[-1]
      else: # If all cards are melds, return highest card from meld with more than 3 cards
        for meld in melds:
          if len(meld) > 3:
            return meld[-1]

  def knock(self): # We decide if it has to knock if our deadwood is < 7.
    non_melds = self.non_melds
    val = []

    for i in range(len(non_melds)):
      val.append(non_melds[i][0])

    if sum(val) < 7:
      return True

    else:
      return False


  def gin(self): #Decide if it is gin
    return len(self.non_melds) == 0


  def decision(self, game): # It decides when we should do each thing
    discard = self.decide_discard(game)


    if self.gin():
      actions_of_game.append(((discard[0], discard[1])))
      return ["gin", discard[0], discard[1]]

    elif self.knock():
      actions_of_game.append(((discard[0], discard[1])))
      return ["knock", discard[0], discard[1]]

    else:
      actions_of_game.append(((discard[0], discard[1])))
      return ["discard", discard[0], discard[1]]


In [6]:
def amazing_solver(game, actions_of_game): # Algorithm to return move
  hand_state = game.get_hand_state() # Get hand for game
  pepe = Hand(game, actions_of_game) # Create object for that state

  if hand_state == 'pick':
    deck_discard = pepe.deck_or_discard() # Returns true for discard, false for deck
    if deck_discard == True:
      return ['discard']
    else:
      return ['deck']
  else:
    pepe = Hand(game, actions_of_game)
    return pepe.decision(game)

In [7]:
game_loop(amazing_solver, GinRummy, 'rummy', multi_player=False, id=None)

Creating new game...
Joining game with id: 9619
Playing as O
Waiting for the other player to join...
X: 0, O: 0
8♤ 9♥ 10♤ 8♧ 5♧ 2♤ 11♦ 11♥ 12♦ 1♤ 
Discard: 8♦
Making next move...
['discard']
b'Valid move'
X: 0, O: 0
8♤ 9♥ 10♤ 8♧ 5♧ 2♤ 11♦ 11♥ 12♦ 1♤ 8♦ 
Discard: 9♦
Making next move...
OUR HAND [(1, 'S'), (2, 'S'), (5, 'C'), (8, 'C'), (8, 'D'), (8, 'S'), (9, 'H'), (10, 'S'), (11, 'D'), (11, 'H'), (12, 'D')]
MELDS [[(8, 'C'), (8, 'D'), (8, 'S')]]
PAST ACTIONS [(0, 'H'), (0, 'C'), (0, 'D'), (0, 'S'), (13, 'H'), (13, 'C'), (13, 'D'), (13, 'S'), (8, 'D'), (9, 'D'), (9, 'D')]
['discard', 11, 'H']
b'Valid move'
X: 0, O: 0
8♤ 9♥ 10♤ 8♧ 5♧ 2♤ 11♦ 12♦ 1♤ 8♦ 
Discard: 3♧
Making next move...
['deck']
b'Valid move'
X: 0, O: 0
8♤ 9♥ 10♤ 8♧ 5♧ 2♤ 11♦ 12♦ 1♤ 8♦ 9♤ 
Discard: 3♧
Making next move...
OUR HAND [(1, 'S'), (2, 'S'), (5, 'C'), (8, 'C'), (8, 'D'), (8, 'S'), (9, 'H'), (9, 'S'), (10, 'S'), (11, 'D'), (12, 'D')]
MELDS [[(8, 'S'), (9, 'S'), (10, 'S')]]
PAST ACTIONS [(0, 'H'), (0, 'C'), (0, 'D'), (